# Zoning Sketchbook

In [1]:
# import standard and 3rd-party libraries
import sys
from shapely.geometry import shape
from shapely.ops import transform
import shapely.wkt
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

ModuleNotFoundError: No module named 'OCC'

In [2]:
# add udtools core modules to path
module_path = '../python/'
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
# import udtools modules
from models.zoninglot import ZoningLot
from models.site import Site
from models.scenario import Scenario

In [4]:
# get query bounds and area centroid from default
aoi = shapely.wkt.loads('POLYGON ((1009405.90154069 234515.39897036, 1009930.58999844 234367.63467916, 1009849.1944143 234052.07026066, 1009314.48803851 234194.82559284, 1009405.90154069 234515.39897036))')

# define common query params for study area
params = {
    'center_x': aoi.representative_point().x,
    'center_y': aoi.representative_point().y,
    'bounds': aoi.wkt
}

## Setup

Define a `Scenario` and `Site` before using them to construct a `ZoningLot`. Calling the `define_zone` method of `ZoningLot` will use `ZoneFactory` to construct a custom `Zone` object using the appropriate subclass (e.g. `ContextualDistrict`) for the provided arguments.

The `Zone` class defines three methods that are used by `ZoningLot` in generating envelopes, new buildings, and instantiating ExistingBuilding objects based on 3d models in the `buildings` table. These are:

- `get_additives`
- `get_subtractices`
- `get_allowables`


In [5]:
my_scenario = Scenario('A')

my_site = Site('2', ['2025540049']) # works
# my_site = Site('1', ['2025770022']) 
# fails on existing building,
# (compound envelopes not yet supported)
# my_site = Site('3', ['2025770020', '2025770022'])
# fails on existing building,
# (compound envelopes not yet supported)
# fails on envelope creation, not sure why

zl = ZoningLot(my_scenario, my_site)
zl.define_zone('R6A')

The `ZoningLot` constructor calls the `load_shape` and `load_edges` methods, which define its shape and topological relationships. In other words, it classifies the lot type (Interior, Through, Corner), the types of each of its edges (Front, Rear, Side) and the street width type for each Front edge (Narrow, Wide).

Incoming geometries from the database are stored in a [well-known text](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) representation. This is interpreted as the OpenCASCADE geometry type appropriate to each operation that requires `ZoningLot` geometry as an argument.

*See ZH 15*

In [6]:
zl.geom # the full zoning lot boundary expressed as wkt
zl.area # geometric area of the zoning lot boundary in square feet
zl.baseplane_elevation # approximate baseplane ht in feet
zl.edges # dictionary of lot edges, classified by type
# zl.type # not yet implemented

{'ids': [1320, 1322, 2514, 2518, 3723],
 'front': [{'id': 1322,
   'geom': 'LINESTRING(-1311.9080969805364 -438.1853605140059,-1345.9346148485783 -522.8711992133467)',
   'azimuth': 3.523647227744336,
   'length': 91.266068143834,
   'front': True,
   'streetname': 'Cypress Avenue',
   'streetwidth': Decimal('80'),
   'baseplane': 46}],
 'rear': [{'id': 2518,
   'geom': 'LINESTRING(-1445.6253606791142 -480.48513468392775,-1410.5712887758855 -397.8964062560408)',
   'azimuth': 0.4013973760000926,
   'length': 89.72004246728818,
   'front': False,
   'streetname': None,
   'streetwidth': None},
  {'id': 3723,
   'geom': 'LINESTRING(-1410.5712887758855 -397.8964062560408,-1409.9751841539983 -396.48924978860305)',
   'azimuth': 0.40070420672465623,
   'length': 1.5282113872390695,
   'front': False,
   'streetname': None,
   'streetwidth': None}],
 'side': [{'id': 2514,
   'geom': 'LINESTRING(-1345.9346148485783 -522.8711992133467,-1445.6253606791142 -480.48513468392775)',
   'azimuth': 5.

In the simplest use case we can fetch the existing buildings on the `ZoningLot` and use them to create a compound solid to use as the envelope by setting the `use_as_envelope` flag. This is useful for estimating existing floor area and evaluating conversions, extensions and enlargements.

In [7]:
bldg_result = zl.load_buildings(use_as_envelope=True)

was compound
<class 'TopoDS_Solid'>
[<class 'TopoDS_Solid'>]


In [8]:
# OCC renderer
renderer = JupyterRenderer()

In [9]:
# display the current envelope, based on the existing building(s)
renderer.DisplayShape(zl.envelope, render_edges=True, topo_level="Face", shape_color="#abdda4", update=True)

In [10]:
# massing_goals are defined as a list of dictionaries 
# defining height, usegroup and an optional label for each floor
# the last element is assumed to repeat upwards until evaluation stops
massing_goals = [
  (11.5, 4, 'school'),
  (9.5, 2, 'residential'),
]

In [11]:
massing = zl.generate_building(massing_goals)

using baseplane
sectioning at 57.0
<class 'TopoDS_Compound'>
sectioning at 66.5
<class 'TopoDS_Compound'>
sectioning at 76.0
<class 'TopoDS_Compound'>
sectioning at 85.5
<class 'TopoDS_Compound'>
sectioning at 95.0
<class 'TopoDS_Compound'>
sectioning at 104.5
<class 'TopoDS_Compound'>
sectioning at 114.0
<class 'TopoDS_Compound'>


In [12]:
for floor in massing.floors:
    # if floor.NbChildren() > 0:
    #     print(floor)
    print(floor.NbChildren())
    renderer.DisplayShape(floor, render_edges=True)


1
1
1
1
1
1
3


In [13]:
# summarize the current proposed building
massing.summarize()

39665.91523443459 sqft in the current building


In [14]:
zl.baseplane_elevation

46

More commonly, you'll want to generate a zoning envelope for a building that doesn't exist yet. To do this, call `generate_envelope` on the `ZoningLot`, passing in the required `massing_goals`. Even though you're not generating the building itself yet, these are needed for the algorithm to be able to process spatial constraints conditioned on per-floor uses, like rear yard depths.

In [15]:
massing_goals = [
  (15.0, 4, 'school'),
  (10.0, 2, 'residential'),
]

zl.baseplane_elevation = 0
zl.generate_envelope(massing_goals)


False
False
curve 1:
(-1311.9080969805364, -438.1853605140059)
(-1311.9080969805364, -438.1853605140059)
(-1345.9346148485783, -522.8711992133467)
(-1345.9346182054674, -522.8711977860796)
...set back by 0
...elevated by 60
curve 2:
(-1321.1871029048277, -434.45708355849854)
(-1321.11264653633, -434.2717751458897)
(-1355.2136207728697, -519.1429222578394)
(-1355.139176840876, -518.9576447974357)
...set back by 10.0
...elevated by 60
curve 3:
(-1321.1871029048277, -434.45708355849854)
(-1321.11264653633, -434.2717751458897)
(-1355.2136207728697, -519.1429222578394)
(-1355.139176840876, -518.9576447974357)
...set back by 10.0
...elevated by 70
curve 4:
(-1311.9080969805364, -438.1853605140059)
(-1311.9080969805364, -438.1853605140059)
(-1345.9346148485783, -522.8711992133467)
(-1345.9346182054674, -522.8711977860796)
...set back by 0
...elevated by 70
<class 'TopoDS_Wire'>
<class 'TopoDS_Wire'>
False
False
curve 1:
(-1445.6253606791142, -480.48513468392775)
(-1445.6253606791142, -480.485

In [16]:
# display the current envelope, based on the existing building(s)
renderer_2 = JupyterRenderer()
renderer_2.DisplayShape(zl.envelope, render_edges=True, topo_level="Face", shape_color="#abdda4", update=True)

In [17]:
# use same massing goals from before
massing = zl.generate_building(massing_goals)

using baseplane
sectioning at 14.5
<class 'TopoDS_Compound'>
sectioning at 24.5
<class 'TopoDS_Compound'>
sectioning at 34.5
<class 'TopoDS_Compound'>
sectioning at 44.5
<class 'TopoDS_Compound'>
sectioning at 54.5
<class 'TopoDS_Compound'>
sectioning at 64.5
<class 'TopoDS_Compound'>


In [18]:
massing.summarize()


44224.88373680335 sqft in the current building


In [19]:
for floor in massing.floors:
  renderer_2.DisplayShape(floor, render_edges=True)

## Export to STEP file

(can be opened with Rhino)

In [20]:
from connectors.step import shapes_to_step


from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID

from geometry.measure import surface_area

explorer = TopExp_Explorer(zl.envelope, TopAbs_SOLID)

solids = []

while explorer.More():
    solids.append(explorer.Current())
    print(surface_area(explorer.Current()))
    explorer.Next()

# renderer.DisplayShape(explorer.Current(), render_edges=True)



shapes_to_step(solids, "/Users/carsten/Desktop/envelope.stp")

47900.59184109822


1